In [1]:
import pandas as pd
#import yfinance as yf
from yahoofinancials import YahooFinancials as yfs
import json
import pandas as pd
import requests
import numpy as np

In [2]:
#pip install yahoofinancials

In [3]:
ticker = 'AMZN'

In [4]:
def Balance(ticker):
    df = pd.read_html('https://www.marketwatch.com/investing/stock/{}/financials/balance-sheet'.format(ticker))
    df1 =  pd.concat([df[4],df[5]])
    maindf = df1
    maindf = maindf.set_index('{}'.format(maindf.columns[0])) #Makes first column the index 
    maindf = maindf.drop(columns = maindf.columns[len(maindf.columns)-1]) # Gets rid of last colsumn 
    maindf = maindf.replace('-',0)
    
    for i in maindf.columns:
        #maindf[i] = maindf[i].replace('B','e9')
        #print(i)
        maindf['{}'.format(i)] = maindf['{}'.format(i)].str.replace('K','e3')
        maindf['{}'.format(i)] = maindf['{}'.format(i)].str.replace('M','e6')
        maindf['{}'.format(i)] = maindf['{}'.format(i)].str.replace('B','e9')
        maindf['{}'.format(i)] = maindf['{}'.format(i)].str.replace('(','-')
        maindf['{}'.format(i)] = maindf['{}'.format(i)].str.replace(')','')
        maindf['{}'.format(i)] = maindf['{}'.format(i)].str.replace('%','')
        maindf['{}'.format(i)] = maindf['{}'.format(i)].str.replace(',','')
    
    for i in maindf.columns: #converts everything to float 
        try:
            maindf['{}'.format(i)] = maindf['{}'.format(i)].astype(float)
        except:
            print(i)

    return maindf 

In [5]:
def Income(ticker):
    #income
    df = pd.read_html('https://www.marketwatch.com/investing/stock/{}/financials'.format(ticker))
    df = df[4]
    maindf = df
    maindf = maindf.set_index('{}'.format(maindf.columns[0])) #Makes first column the index 
    maindf = maindf.drop(columns = maindf.columns[len(maindf.columns)-1]) # Gets rid of last colsumn 
    maindf = maindf.replace('-',0)

    for i in maindf.columns:
        #maindf[i] = maindf[i].replace('B','e9')
        #print(i)
        maindf['{}'.format(i)] = maindf['{}'.format(i)].str.replace('K','e3')
        maindf['{}'.format(i)] = maindf['{}'.format(i)].str.replace('M','e6')
        maindf['{}'.format(i)] = maindf['{}'.format(i)].str.replace('B','e9')
        maindf['{}'.format(i)] = maindf['{}'.format(i)].str.replace('(','-')
        maindf['{}'.format(i)] = maindf['{}'.format(i)].str.replace(')','')
        maindf['{}'.format(i)] = maindf['{}'.format(i)].str.replace('%','')
        maindf['{}'.format(i)] = maindf['{}'.format(i)].str.replace(',','')
    
    for i in maindf.columns: #converts everything to float 
        try:
            maindf['{}'.format(i)] = maindf['{}'.format(i)].astype(float)
        except:
            print(i)
    return(maindf) 

In [6]:
#cashflow
def cash(ticker):
    df = pd.read_html('https://www.marketwatch.com/investing/stock/{}/financials/cash-flow'.format(ticker))
    df1 =  pd.concat([df[4],df[5],df[6]])
    maindf = df1
    maindf = maindf.set_index('{}'.format(maindf.columns[0])) #Makes first column the index 
    maindf = maindf.drop(columns = maindf.columns[len(maindf.columns)-1]) # Gets rid of last colsumn 
    maindf = maindf.replace('-',0)
    
    for i in maindf.columns:
        #maindf[i] = maindf[i].replace('B','e9')
        #print(i)
        maindf['{}'.format(i)] = maindf['{}'.format(i)].str.replace('K','e3')
        maindf['{}'.format(i)] = maindf['{}'.format(i)].str.replace('M','e6')
        maindf['{}'.format(i)] = maindf['{}'.format(i)].str.replace('B','e9')
        maindf['{}'.format(i)] = maindf['{}'.format(i)].str.replace('(','-')
        maindf['{}'.format(i)] = maindf['{}'.format(i)].str.replace(')','')
        maindf['{}'.format(i)] = maindf['{}'.format(i)].str.replace('%','')
        maindf['{}'.format(i)] = maindf['{}'.format(i)].str.replace(',','')
    
    for i in maindf.columns: #converts everything to float 
        try:
            maindf['{}'.format(i)] = maindf['{}'.format(i)].astype(float)
        except:
            print(i)

    return maindf 

In [7]:
def DATA(ticker):
    z = cash(ticker)

    #Free CashFlow
    #FCF = z.loc[['Net Income before Extraordinaries  Net Income before Extraordinaries','Capital Expenditures  Capital Expenditures','Funds from Operations  Funds from Operations','Free Cash Flow  Free Cash Flow']]
    FCF = z.loc[['Net Income before Extraordinaries  Net Income before Extraordinaries','Free Cash Flow  Free Cash Flow']]
    df = pd.DataFrame()
    df['FCF'] = FCF.loc['Free Cash Flow  Free Cash Flow'] 
    df['Net Income'] = FCF.loc['Net Income before Extraordinaries  Net Income before Extraordinaries']
    minimum = (df['FCF']/df['Net Income']).min()
    #df
    
    #WACC = Wd * Rd * (1 - t) + We + Re
    x = Income(ticker)
    INCOME2 = x.loc[['Pretax Income  Pretax Income','Income Tax  Income Tax']]
    t = x.loc['Pretax Income  Pretax Income']/ x.loc['Income Tax  Income Tax']
    t = t[-1]/ 100 


    bond = pd.read_html('https://finance.yahoo.com/bonds')
    Rf = float(bond[0]['Last Price'][2])
    try: 
        beta = float(yahoo_financials.get_beta())
    except:
        beta = 0
    Rm = float(10.0)
    Ra = Rf + beta * (Rm - Rf)
    print(Ra)
    print(Rf)


    y = Balance(ticker)
    Balance2 = y.loc[['Long-Term Debt  Long-Term Debt','Short Term Debt  Short Term Debt']]
    Balance2 = Balance2.fillna(0)
    #display(Balance2)
    Total_debt = Balance2.sum()[-1] 
    try:
        Balance2 =   (x.loc['Interest Expense  Interest Expense'][-1]) / Total_debt
    except:
        Balance2 =   (x.loc['Interest Expense  Interest Expense'][-2]) / Total_debt
    #Balance2 =   -(x.loc['Interest Expense  Interest Expense'][-1]) / Total_debt
    Rd = (1-Balance2) * t #  Rd * (1 - t)

    
    yahoo_financials = yfs(ticker)
    stock = yahoo_financials.get_key_statistics_data()[ticker]
    Market_Cap = yahoo_financials.get_market_cap() # MarketCap
    Total =  Total_debt + Market_Cap 
    Wd = Total_debt/ Total 
    We = Market_Cap/ Total 
    Re = Ra
    Required_Rate = (Wd * Rd + We * Re)/100
    Required_Rate
    #print(Wd)
    #print(Rd)
    #print(We)
    #print(Re)

    #print(Ra)
    #print(Rf)
    #print(Required_Rate)
    return  DCF_VALUE(Required_Rate,minimum, df,stock), DCF_VALUE_CON(df,stock)

#WACC = Wd * Rd * (1 - t) + We + Re
x = Income(ticker)
INCOME2 = x.loc[['Pretax Income  Pretax Income','Income Tax  Income Tax']]
t = x.loc['Pretax Income  Pretax Income']/ x.loc['Income Tax  Income Tax']
t = t[-1]/ 100 


bond = pd.read_html('https://finance.yahoo.com/bonds')
Rf = float(bond[0]['Last Price'][2])
try: 
    beta = float(yahoo_financials.get_beta())
except:
    beta = 0
Rm = float(10.0)
Ra = Rf + beta * (Rm - Rf)
print(Ra)
print(Rf)


y = Balance(ticker)
Balance2 = y.loc[['Long-Term Debt  Long-Term Debt','Short Term Debt  Short Term Debt']]
Balance2 = Balance2.fillna(0)
#display(Balance2)
Total_debt = Balance2.sum()[-1] 
try:
    Balance2 =   (x.loc['Interest Expense  Interest Expense'][-1]) / Total_debt
except:
    Balance2 =   (x.loc['Interest Expense  Interest Expense'][-2]) / Total_debt
#Balance2 =   -(x.loc['Interest Expense  Interest Expense'][-1]) / Total_debt
Rd = (1-Balance2) * t #  Rd * (1 - t)


yahoo_financials = yfs(ticker)
stock = yahoo_financials.get_key_statistics_data()[ticker]
Market_Cap = yahoo_financials.get_market_cap() # MarketCap
Total =  Total_debt + Market_Cap 
Wd = Total_debt/ Total 
We = Market_Cap/ Total 
Re = Ra
Required_Rate = (Wd * Rd + We * Re)/100
Required_Rate
print(Wd)
print(Rd)
print(We)
print(Re)

print(Ra)
print(Rf)
print(Required_Rate)

In [11]:
def DCF_VALUE(Required_Rate,minimum, df,stock)::
    outstanding_shares = stock['sharesOutstanding']

    Required_rate = Required_Rate
    Pre_rate = .025
    Cashflowgrowthrate = minimum

    years = [1,2,3,4]
    freecash_flow = df['FCF'].values

    terminalvalue = freecash_flow[-1] * (1+Pre_rate)/(Required_rate - Pre_rate)

    futurefreecashflow = []
    discountfactor = []
    discountfuturefreecashflow = []
    for  year in years:
        cashflow = freecash_flow[-1] * (1+ Cashflowgrowthrate) ** year 
        futurefreecashflow.append(cashflow)
        discountfactor.append((1+ Required_rate)**year)


    for i in range(0,len(years)):
        discountfuturefreecashflow.append(futurefreecashflow[i]/discountfactor[i])

    print(discountfuturefreecashflow)
    dis = terminalvalue/(1+Required_rate)**len(years)
    discountfuturefreecashflow.append(dis)
    value = sum(discountfuturefreecashflow)
    fairvalue = value/outstanding_shares

    print(ticker)
    print(fairvalue)
    print(yahoo_financials.get_current_price())

    print(Cashflowgrowthrate)
    print(Required_Rate)
#print(stock.get_current_price())
DCF_VALUE()

[56489936657.87094, 123113925293.60612, 268313959935.86673, 584762291712.9487]
AMZN
-3717.2977928053583
3074.96
1.2151898734177216
0.016423896290333983


In [12]:
def DCF_VALUE_CON(df,stock):
    outstanding_shares = stock['sharesOutstanding']

    Required_rate = .075
    Pre_rate = .025
    Cashflowgrowthrate = .035

    years = [1,2,3,4]
    freecash_flow = df['FCF'][1:5].values

    terminalvalue = freecash_flow[-1] * (1+Pre_rate)/(Required_rate - Pre_rate)

    futurefreecashflow = []
    discountfactor = []
    discountfuturefreecashflow = []
    for  year in years:
        cashflow = freecash_flow[-1] * (1+ Cashflowgrowthrate) ** year 
        futurefreecashflow.append(cashflow)
        discountfactor.append((1+ Required_rate)**year)


    for i in range(0,len(years)):
        discountfuturefreecashflow.append(futurefreecashflow[i]/discountfactor[i])

    print(discountfuturefreecashflow)
    dis = terminalvalue/(1+Required_rate)**len(years)
    discountfuturefreecashflow.append(dis)
    value = sum(discountfuturefreecashflow)
    fairvalue = value/outstanding_shares

    #print(ticker)
    #print(fairvalue)
    #print(yahoo_financials.get_current_price())

    #print(Cashflowgrowthrate)
    #print(Required_rate)
#print(stock.get_current_price())
DCF_VALUE_CON()

[24955534883.72093, 24026956841.535965, 23132930540.455555, 22272170334.29907]
AMZN
978.6684733637893
3074.96
0.035
0.075


In [13]:
df = pd.read_html('https://finance.yahoo.com/quote/AAPL/financials?p=AAPL')

ValueError: No tables found

In [ ]:
df